In [1]:
!pip install scapy
!pip install IP2Location    # can also be used in bonus but is very large

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scapy: filename=scapy-2.5.0-py2.py3-none-any.whl size=1444325 sha256=2533a0b26f068ef90f2408d7e5d4e67ce21676a3fb9ac5a4fcab0fe20c709d1f
  Stored in directory: /root/.cache/pip/wheels/82/b7/03/8344d8cf6695624746311bc0d389e9d05535ca83c35f90241d
Successfully built scapy
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from scapy.all import *
from bs4 import BeautifulSoup
import requests

In [3]:
ANTARCTICA_list = []

x='31.187.93.'
for i in range(96, 112):
  ANTARCTICA_list.append(x+str(i))

x='46.136.153.'
for i in range(0, 256):
  ANTARCTICA_list.append(x+str(i))

x='46.136.231.'
for i in range(0, 256):
  ANTARCTICA_list.append(x+str(i))

x='46.136.176.'
for i in range(0, 256):
  ANTARCTICA_list.append(x+str(i))

In [ ]:
# A bonus function to get the location of the attacker
def bonus_location(ip):
    print(ip)
    response = requests.get("https://geolocation-db.com/json/"+ip+"&position=true").json()
    return(response['country_name'] + " - " + response['city'] + " - " + response['postal'])

# Function to detect port scanning techniques
def detect_port_scans(pkt, x):
    # Check if packet is a TCP packet
    if pkt.haslayer(TCP):
        # Check if TCP flags are set to indicate a port scan
        if pkt['TCP'].flags == 0x002: # SYN
            print(f'Packet #{x}:')
            print(f"Stealth Port Scan detected from {pkt['IP'].src} in {bonus_location(pkt[IP].src)} to {pkt['IP'].dst}")
            print('----------------')
        elif pkt['TCP'].flags == 0x000: # NULL
            print(f'Packet #{x}:')
            print(f"NULL Port Scan detected from {pkt['IP'].src} in {bonus_location(pkt[IP].src)} to {pkt['IP'].dst}")
            print('----------------')
        elif pkt['TCP'].flags == 0x001: # FIN
            print(f'Packet #{x}:')
            print(f"FIN Port Scan detected from {pkt['IP'].src} in {bonus_location(pkt[IP].src)} to {pkt['IP'].dst}")
            print('----------------')
        elif pkt['TCP'].flags == 0x029: # FIN, PSH, URG
            print(f'Packet #{x}:')
            print(f"Xmas Port Scan detected from {pkt['IP'].src} in {bonus_location(pkt[IP].src)} to {pkt['IP'].dst}")
            print('----------------')

def bonus_scans(pkt, x):
    if pkt.haslayer(TCP):
        if pkt['TCP'].flags == 0x020: # URG
            print(f'Packet #{x}:')
            print(f"URG Port Scan detected from {pkt['IP'].src} in {bonus_location(pkt[IP].src)} to {pkt['IP'].dst}")
            print('----------------')
        elif pkt['TCP'].flags == 0x008: # PSH
            print(f'Packet #{x}:')
            print(f"PSH Port Scan detected from {pkt['IP'].src} in {bonus_location(pkt[IP].src)} to {pkt['IP'].dst}")
            print('----------------')
        elif pkt['TCP'].flags == 0x028: # URG-PSH
            print(f'Packet #{x}:')
            print(f"URG-PSH Port Scan detected from {pkt['IP'].src} in {bonus_location(pkt[IP].src)} to {pkt['IP'].dst}")
            print('----------------')

# Function to detect Nikto scanner
def detect_nikto(pkt, x):
    # Check if packet contains "nikto" in raw payload
    if Raw in pkt and "nikto" in str(pkt[Raw].load):
        print(f'Packet #{x}:')
        print(f"Nikto Scanner detected from {pkt[IP].src} in {bonus_location(pkt[IP].src)} to {pkt[IP].dst}")
        print('----------------')

# Function to detect attackers from ANTARCTICA
def detect_antarctica(pkt, x):
    if (IP in pkt) and (pkt[IP].src in ANTARCTICA_list):
      print(f'Packet #{x}:')
      print(f"Attack from ANTARCTICA detected from {pkt[IP].src} to {pkt[IP].dst}")
      print('----------------')

# Main function to read packets from pcap file and call detection functions
def main():
    files = ['/content/stealth.pcapng', '/content/null.pcap', '/content/fin.pcap', '/content/xmas.pcap', '/content/nikto.pcap', '/content/Antarctica.pcap']
    files = ['/content/Antarctica.pcap']
    
    for filename in files:
        print(f"Reading packets from {filename}")
        packets = rdpcap(filename)
        # Call detection functions for each packet
        x=1
        for pkt in packets:
            #detect_port_scans(pkt, x)
            detect_nikto(pkt, x)
            detect_antarctica(pkt, x)
            x = x+1

main()

In [ ]:
# Code to extract Nikto Scanner packets' numbers

packets = rdpcap('/content/nikto.pcap')
x=1
for pkt in packets:
  if Raw in pkt and "nikto" in str(pkt[Raw].load):
        print(x)
        #print(pkt[Raw])
        #print(pkt[Raw].load)
        #print(f"Nikto Scanner detected from {pkt[IP].src} to {pkt[IP].dst}")
  x=x+1

926
Raw
b'GET / HTTP/1.1\r\nHost: 192.168.1.215\r\nOrigin: nikto.example.com\r\nConnection: Keep-Alive\r\nUser-Agent: Mozilla/5.00 (Nikto/2.1.6) (Evasions:None) (Test:origin_reflection)\r\n\r\n'
Nikto Scanner detected from 192.168.1.218 to 192.168.1.215
1767
Raw
b'PUT /nikto-test-EJcrh26r.html HTTP/1.1\r\nUser-Agent: Mozilla/5.00 (Nikto/2.1.6) (Evasions:None) (Test:put_del_test: PUT)\r\nContent-Length: 22\r\nConnection: Keep-Alive\r\nContent-Type: application/x-www-form-urlencoded\r\nHost: 192.168.1.215\r\n\r\nThis was a Nikto test.'
Nikto Scanner detected from 192.168.1.218 to 192.168.1.215


In [4]:
# Bonus verification

link = "https://www.countryipblocks.net/search_ip.php?search_ip=192.168.1.218&search_ip_result="
response = requests.get(link)
soup = BeautifulSoup(response.content)
print(soup)
elems = soup.find_all('h1', class_="mains")
print(elems)
for elem in elems:
    print(elem.text[:23])
    if elem.text[:23] == "IP Address assigned to":
        print(elem.text[25:])


<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>403 Forbidden</title>
</head><body>
<h1>Forbidden</h1>
<p>You don't have permission to access this resource.</p>
<p>Additionally, a 403 Forbidden
error was encountered while trying to use an ErrorDocument to handle the request.</p>
</body></html>

[]
